In [2]:
# Import necessary libraries
import fastf1 as ff1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

years = [2018, 2019, 2020, 2021, 2022, 2023, 2024] # Year of grand prix list
# Get circuit length (in meters)
circuits_length = {
    'Melbourne': 5278,          # Albert Park Circuit
    'Shanghai': 5451,           # Shanghai International Circuit
    'Suzuka': 5807,             # Suzuka International Racing Course
    'Sakhir': 5412,             # Bahrain International Circuit (Bahrain è il paese, Sakhir la città)
    'Jeddah': 6174,             # Jeddah Corniche Circuit
    'Miami': 5412,              # Miami International Autodrome
    'Imola': 4909,              # Autodromo Enzo e Dino Ferrari
    'Monte Carlo': 3337,        # Circuit de Monaco (Monaco è il paese, Monte Carlo la zona)
    'Barcelona': 4657,          # Circuit de Barcelona-Catalunya
    'Montréal': 4361,           # Circuit Gilles Villeneuve
    'Spielberg': 4318,          # Red Bull Ring (Austria)
    'Silverstone': 5891,        # Silverstone Circuit
    'Spa-Francorchamps': 7004,  # Circuit de Spa-Francorchamps (Belgio)
    'Budapest': 4381,           # Hungaroring
    'Zandvoort': 4259,          # Circuit Zandvoort
    'Monza': 5793,              # Autodromo Nazionale Monza
    'Baku': 6003,               # Baku City Circuit
    'Singapore': 4940,          # Marina Bay Street Circuit
    'Austin': 5513,             # Circuit of the Americas
    'Mexico City': 4304,        # Autódromo Hermanos Rodríguez
    'São Paulo': 4309,          # Autódromo José Carlos Pace (Interlagos)
    'Las Vegas': 6201,          # Las Vegas Strip Circuit
    'Lusail': 5419,             # Losail International Circuit (Qatar)
    'Yas Marina': 5281,         # Yas Marina Circuit
    'Yas Island': 5281,         # Yas Marina Circuit
    'Le Castellet': 5842,       # Circuit Paul Ricard
    'Sochi': 5848,              # Sochi Autodrom
    'Portimão': 4653,           # Autódromo Internacional do Algarve
    'Istanbul': 5338,           # Istanbul Park
    'Nürburgring': 5148,        # Nürburgring
    'Hockenheim': 4574,         # Hockenheimring
    'Adelaide': 3780,           # Adelaide Street Circuit
    'Phoenix': 3770,            # Phoenix Street Circuit
    'Estoril': 4415,            # Autódromo do Estoril
    'Magny-Cours': 4411,        # Circuit de Nevers Magny-Cours
    'Indianapolis': 4180,       # Indianapolis Motor Speedway
    'Kuala Lumpur': 5543,       # Sepang International Circuit (Malaysia)
    'Valencia': 4005,           # Circuit Ricardo Tormo
    'Yeongam': 5603,            # Korean International Circuit
    'Jerez de la Frontera': 4423,  # Circuit de Jerez
    'Kyalami': 4300,            # Kyalami Grand Prix Circuit
    'Donington': 4023,          # Donington Park Circuit
    'Oyama': 4571,              # Fuji Speedway (città più vicina)
    'Mugello':5245              # Autodromo internazionale del Mugello
}

max_database_tyre_meters = 0
max_database_distance = 0

year_distance = []

for year in years:
    try:
        # circuit_names = list(circuits_length.keys())
        calendario = ff1.get_event_schedule(year)
    except Exception as e:
        print(f"Could not load schedule for year {year}: {str(e)}")
        continue

    # Filtra solo le gare convenzionali (esclude test, sprint shootout, etc.)
    gare_ufficiali = calendario[calendario['EventFormat'] == 'conventional']

    # Estrai i nomi dei Gran Premi
    lista_gran_premi = gare_ufficiali['EventName'].tolist()

    max_year_tire_meters = 0
    max_year_distance = 0
    
    for race in lista_gran_premi:

        # Load the session data for the specified race
        session = ff1.get_session(year, race, 'R')  # Example: Monaco 2023 Race
        session.load()
        location = session.event.Location

        circuit_length = circuits_length.get(location, 0)  # Get the circuit length in meters

        if session.drivers:
            drivers = session.drivers # Get the list of drivers

            for target_driver in drivers:
                # Get the laps data for the target driver
                driver_laps = session.laps.pick_driver(target_driver)

                # Check if driver_laps is not empty before accessing data
                if not driver_laps.empty:
                    # Get driver name from the first lap data since all laps have the same driver info
                    driver_name = driver_laps.iloc[0]['Driver']
                else:
                    # Skip this driver if no lap data is available
                    print(f"No lap data for driver {target_driver} in session {session.event.EventName} ({year})")
                    continue

                # Prepare the data for analysis
                tire_data = []
                tire_life_data = []
                driver_meters = 0
                live_tires=0

                # Lista per salvare i risultati
                #cumulative_meters = []
                #total_meters = 0  # Distanza cumulativa

                for lap in driver_laps.iterlaps():
                    lap_number = lap[1]['LapNumber']
                    lap_time = lap[1]['LapTime']
                    compound = lap[1]['Compound']
                    tire_life = lap[1]['TyreLife']
                    fresh_tire = lap[1]['FreshTyre']
                    stint = lap[1]['Stint']

                    #car_data = lap.get_car_data()  # Alternativa: .get_pos_data()

                    # Calculate meters driven on these tires
                    # Add this lap's distance to the cumulative tire distance
                    if(tire_life and tire_life > 0):

                        cumulative_tire_distance = int(tire_life) * circuit_length
                        driver_meters = int(lap_number) * circuit_length
                    
                        if((tire_data and cumulative_tire_distance <= tire_data[-1]['TireMeters'])):
                            live_tire = tire_data[-1]['TireMeters']
                        
                            tire_life_data.append({
                                'DriverID': target_driver,
                                'Driver': driver_name,
                                'GranPrix': race,
                                'Stint': int(tire_data[-1]['Stint']),
                                'Compound': tire_data[-1]['Compound'],
                                'TireLife': tire_data[-1]['TireLife'],
                                'TireMeters': live_tire,
                            })
                    
                        if(driver_laps.iloc[-1]['LapNumber'] == int(lap_number)):
                        
                            tire_life_data.append({
                                'DriverID': target_driver,
                                'Driver': driver_name,
                                'GranPrix': race,
                                'Stint': stint,
                                'Compound': compound,
                                'TireLife': tire_life,
                                'TireMeters': cumulative_tire_distance,
                            })
                    else:
                        cumulative_tire_distance = 0
                        driver_meters = 0

                    #if not car_data.empty:
                        # La distanza massima registrata nel giro
                        #lap_distance_meters = car_data['Distance'].max()
                        #total_meters += lap_distance_meters

                    tire_data.append({
                        'DriverID': target_driver,
                        'Driver': driver_name,
                        'LapNumber': lap_number,
                        'Stint': stint,
                        'TireLife': tire_life,
                        'TireMeters': cumulative_tire_distance,
                        'Compound': compound,
                        'IsFreshTire': fresh_tire,
                        'DriverMeters': driver_meters,
                        'LapTime': lap_time,
                    })

                    # Create DataFrame
                    tire_df = pd.DataFrame(tire_data)
                    # Save to CSV
                    #tire_df.to_csv(f'{target_driver}_{race}_location_{location}_{year}_tire_data.csv', index=False)
                
                tire_life_df = pd.DataFrame(tire_life_data)
                #tire_life_df.to_csv(f'{target_driver}_{race}_lives_{location}_{year}_tire_data.csv', index=False)
        
            # Print the max total distance driven by the drivers in the session
            #qui devo normalizzare con il massimo del massimo o per i metri dello stint devo prendere lo stint più lungo, bisogna mantenere le proporzioni tra le due colonne?
            max_tire_meters = tire_life_df['TireMeters'].max() if not tire_life_df.empty else 0
            print(f"max_tyre: {max_tire_meters} meters")
            if(max_tire_meters > max_year_tire_meters):
                max_year_tire_meters = max_tire_meters

            total_laps = session.total_laps
            total_meters = total_laps * circuit_length
            if(total_meters > max_year_distance):
                max_year_distance = total_meters
        else:
            print(f"Session {location} in {year} has no drivers data.")
            continue

    year_distance.append({
        'Year': year,
        'Stint': max_year_tire_meters,
        'Distance': max_year_distance
        
    })

    if(max_year_tire_meters > max_database_tyre_meters):
        max_database_tyre_meters = max_year_tire_meters #update max dataset tire meters

    if(max_year_distance > max_database_distance):
        max_database_distance = max_year_distance #update max dataset distance

for y in year_distance:
    print(f"year:{y['Year']}")
    print(f"Max distance {y['Distance']} meters")
    print(f"Max stint {y['Stint']} meters")

print(f"Max distance in database: {max_database_distance} meters; stint: {max_database_tyre_meters} meters")

print("this all done")

core           INFO 	Loading data for Australian Grand Prix - Race [v3.4.5]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55

max_tyre: 21112 meters


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 27: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  2: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  9: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 31: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 55: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 16: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  8: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 18: Lap timing integrity check failed for 1 lap(s)
core        WARNING 

max_tyre: 0 meters


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '77', '7', '44', '33', '27', '14', '5', '55', '20', '31', '11', '2', '18', '35', '9', '8', '10', '16', '28']
c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\core.py:3022: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will b

...

core        WARNING 	Driver 63: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver 44: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 55: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 16: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver  1: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  4: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 81: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 30: Lap timing integrity check failed for 2 lap(s)
core        WARNING 	Driver 77: Lap timing integrity check failed for 2 lap(s)
Request for URL https://ergast.com/api/f1/2024/22/laps/1.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\requests_cache\session.py", line 286, in _resend
    response = self._send_and_cache(requ

max_tyre: 62010 meters


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
Request for URL https://ergast.com/api/f1/2024/24/laps/1.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\requests_cache\session.py", line 286, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\requests_cache\session.py", line 254, in _send_and_cache
    response = super().send(request, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\

max_tyre: 5281 meters
year:2018
Max distance 309944 meters
Max stint 245295 meters
year:2019
Max distance 309944 meters
Max stint 211582 meters
year:2020
Max distance 470844 meters
Max stint 149024 meters
year:2021
Max distance 309944 meters
Max stint 195084 meters
year:2022
Max distance 309626 meters
Max stint 175240 meters
year:2023
Max distance 310050 meters
Max stint 292248 meters
year:2024
Max distance 310050 meters
Max stint 238136 meters
Max distance in database: 470844 meters; stint: 292248 meters
this all done


c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\core.py:3022: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\core.py:3022: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\core.py:3022: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
